In [ ]:
import sys
from pathlib import Path
# code_path = Path('../../').resolve()
# sys.path.append(str(code_path))

In [ ]:
%load_ext autoreload
%autoreload 2
import _path
import powerlaw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.stats as stats
import queue
import seaborn
import random
from scipy.optimize import leastsq
from rw_data_processing import *
from Data_synthesize import *
from transition_probability_estimation import *
from plot_results import *
from firefly_optimizer import *
plt.style.use("../rw_visualization.mplstyle")


In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Color
current_palette = seaborn.color_palette()
current_palette

# 1. Cheng2020

## 1.0 Lode main simulation restuls

In [ ]:
data_path = Path('../synthetic_data_results_cheng2020')
demographic_data_list, social_data_list, course_of_disease_data_list, contact_data_list, transmission_digraph_list = \
    load_synthetic_data(data_path, return_len=100)

In [ ]:
demographic_data_all = np.concatenate(demographic_data_list, axis=0)
social_data_all = np.concatenate(social_data_list, axis=0)
contact_data_all = np.concatenate(contact_data_list, axis=0)
course_of_disease_data_all = np.concatenate(course_of_disease_data_list, axis=0)
transmission_digraph_all = np.concatenate(transmission_digraph_list, axis=0)


## 1.1 Cheng2020 Figure 2

In [ ]:
household_lower_bound = np.array([1.6, 1.4, 3, 0, 0, 0])/100
household_upper_bound = np.array([9.8, 16.8, 56.4, 49, 65.7, 100])/100
household_range = household_upper_bound-household_lower_bound
household_weight = 1/household_range
norm_household_weight = household_weight/max(household_weight)
print('household_weight: ', norm_household_weight)

In [ ]:
health_care_lower_bound = np.array([0.2, 0.7, 0.5, 0, 0, 0])/100
health_care_upper_bound = np.array([3, 5.7, 13.5, 18.4, 3.3, 2.6])/100
health_care_range = health_care_upper_bound-health_care_lower_bound
health_care_weight = 1/health_care_range
norm_health_care_weight = health_care_weight/max(health_care_weight)
print('health_care_weight: ', norm_health_care_weight)

In [ ]:
other_lower_bound = np.array([0, 0, 0.1, 0, 0, 0])/100
other_upper_bound = np.array([1, 0.6, 3.3, 4.2, 1.1, 3.3])/100
other_range = other_upper_bound-other_lower_bound
other_weight = 1/other_range
norm_other_weight = other_weight/max(other_weight)
print('other_weight: ', norm_other_weight)

In [ ]:
total_household_contact_numbers = []
for i in range(100):
    tmp = []
    for contact_data in contact_data_all[i*100:(i+1)*100]:
        tmp.append(len(contact_data['household_contacts_matrix']))
    total_household_contact_numbers.append(sum(tmp))

In [ ]:
# Household
layer = 'Household'
ax1, ax2, (household_mean_contact_number, household_contact_lb, household_contact_ub, household_mean_infection_case, household_infection_lb, household_infection_ub, household_mean_attack_rate, household_attack_lb, household_attack_ub) = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100, layer=layer, save_fig=False)
ax1.set_ylim(0, 150)
ax1.set_ylabel('Household contacts')
ax1.set_xlabel("")
ax2.set_ylim(0, 65)
ax2.set_ylabel("")
# ax1.set_title('Synthetic data')
# plt.savefig('RW2022_synthesis_%s_contact_bar_chart.pdf' % layer)

ax1, ax2, (household_contacts, household_infected_contacts, household_attack_rate, household_attack_rate_lb, household_attack_rate_upper_bound) = plot_cheng2020_bar_chart(layer='Household', save_fig=False)
ax1.set_ylim(0, 150)
# ax1.set_ylabel('Household contacts')
ax1.set_xlabel("")
ax1.set_ylabel("")
ax2.set_ylim(0, 65)
# ax1.set_title('Cheng2020')
# plt.savefig('RW2022_cheng_%s_bar_chart.pdf' % layer)

In [ ]:
# Health care
layer = 'Health care'
ax1, ax2, ((health_care_mean_contact_number, health_care_contact_lb, health_care_contact_ub, health_care_mean_infection_case, health_care_infection_lb, health_care_infection_ub, health_care_mean_attack_rate, health_care_attack_lb, health_care_attack_ub)) = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100,
    layer=layer, save_fig=False)
ax1.set_ylim(0, 400)
ax1.set_ylabel('Health care contacts')
ax1.set_xlabel("")
ax2.set_ylim(0, 10)
ax2.set_ylabel("")
# ax1.set_title('Synthetic data')
plt.savefig('RW2022_synthesis_%s_contact_bar_chart.pdf' % layer.replace(' ', '_'))


ax1, ax2, (health_care_contacts, health_care_infected_contacts, health_care_attack_rate, health_care_attack_rate_lb, health_care_attack_rate_upper_bound) = plot_cheng2020_bar_chart(layer='Health care', save_fig=False)
ax1.set_ylim(0, 400)
ax1.set_xlabel("")
ax2.set_ylim(0, 10)
# ax1.set_title('Cheng2020')
# plt.savefig('RW2022_cheng_%s_bar_chart.pdf' % layer.replace(' ', '_'))


In [ ]:
# School
ax1, ax2, _ = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100,
    layer='School', save_fig=False)
ax1.set_ylabel('School contacs')
ax1.set_ylim([0, 100])


In [ ]:
# Workplace
ax1, ax2, _ = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100,
    layer='Workplace', save_fig=False)
ax1.set_ylabel('Workplace contacts')


In [ ]:
# All
layer = 'All'
ax1, ax2, (all_mean_contact_number, all_contact_lb, all_contact_ub, all_mean_infection_case, \
           all_infection_lb, all_infection_ub, all_mean_attack_rate, all_attack_lb, all_attack_ub) = \
    plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=1000, num_source_cases=100,
    layer=layer, save_fig=False)
ax1.set_ylim(0, 1200)
ax2.set_ylim(0, 10)
ax1.set_ylabel('All contacts')
ax2.set_ylabel("")
# ax1.set_title('Synthetic data')
plt.savefig('RW2022_synthesis_%s_contact_bar_chart.pdf' % layer)

ax1, ax2, (all_contacts, all_infected_contacts, all_attack_rate, all_attack_rate_lb, all_attack_rate_upper_bound) = plot_cheng2020_bar_chart(layer='All', save_fig=False)
ax1.set_ylim(0, 1200)
ax2.set_ylim(0, 10)
# ax1.set_title('Cheng2020')
# plt.savefig('RW2022_cheng_%s_bar_chart.pdf' % layer)


## 1.2 Combine plot

In [ ]:
# cheng_infected_contacts_ticks = np.arange(6)-0.15
# covsyn_infected_contacts_ticks = np.arange(6)-0.05
# cheng_attack_rate_ticks = np.arange(6)
# covsyn_attack_rate_ticks = np.arange(6)
# cheng_contacts_ticks = np.arange(6)+0.05
# covsyn_contacts_ticks = np.arange(6)+0.15

In [ ]:
# # Household
# plt.figure(figsize=(10, 8))
# plt.bar(cheng_infected_contacts_ticks, household_infected_contacts, width=0.1)
# plt.bar(covsyn_infected_contacts_ticks, household_mean_infection_case, width=0.1)
# plt.bar(cheng_contacts_ticks, household_contacts, width=0.1)
# plt.bar(covsyn_contacts_ticks, household_mean_contact_number, width=0.1)
# plt.plot(cheng_attack_rate_ticks, household_attack_rate, '--.')
# plt.plot(covsyn_attack_rate_ticks, household_mean_attack_rate, '--.')

In [ ]:
# # Household comparison plot
# fig, ax1 = plt.subplots(figsize=(10, 6))
# ax2 = ax1.twinx()

# # Define x positions
# x = np.arange(6)
# width = 0.15

# # Define colors (ColorBrewer palette)
# cheng_color = "#1f78b4"     # Blue
# covsyn_color = "#a6cee3"    # Light blue
# cheng_inf_color = "#33a02c" # Green  
# covsyn_inf_color = "#b2df8a" # Light green
# attack_cheng_color = "#e31a1c" # Red
# attack_covsyn_color = "#fb9a99" # Light red

# # Define x positions
# cheng_infected_contacts_ticks = x - 2*width
# covsyn_infected_contacts_ticks = x - width
# cheng_contacts_ticks = x + width
# covsyn_contacts_ticks = x + 2*width
# cheng_attack_rate_ticks = x - 0.05
# covsyn_attack_rate_ticks = x + 0.05

# # Plot bars - Contacts
# ax1.bar(cheng_contacts_ticks, household_contacts, width=width, 
#        color=cheng_color, label='Cheng et al. contacts', alpha=0.9)
# ax1.bar(covsyn_contacts_ticks, household_mean_contact_number, width=width,
#        color=covsyn_color, label='CovSyn contacts', alpha=0.9)

# # Plot contacts uncertainty for CovSyn
# y_lower = household_mean_contact_number - household_contact_lb
# y_upper = household_contact_ub - household_mean_contact_number
# ax1.errorbar(covsyn_contacts_ticks, household_mean_contact_number, 
#             yerr=[y_lower, y_upper], fmt='.k', ecolor='black', 
#             capsize=5, linewidth=1, markeredgewidth=1)

# # Plot bars - Infected contacts
# ax1.bar(cheng_infected_contacts_ticks, household_infected_contacts, width=width,
#        color=cheng_inf_color, label='Cheng et al. infected', alpha=0.9)
# ax1.bar(covsyn_infected_contacts_ticks, household_mean_infection_case, width=width,
#        color=covsyn_inf_color, label='CovSyn infected', alpha=0.9)

# # Plot infected contacts uncertainty for CovSyn
# y_lower = household_mean_infection_case - household_infection_lb
# y_upper = household_infection_ub - household_mean_infection_case
# y_upper[y_upper < 0] = 0
# ax1.errorbar(covsyn_infected_contacts_ticks, household_mean_infection_case, 
#             yerr=[y_lower, y_upper], fmt='.k', ecolor='black', 
#             capsize=5, linewidth=1, markeredgewidth=1)

# # Plot attack rates
# ax2.plot(cheng_attack_rate_ticks, household_attack_rate, 'o-', 
#         color=attack_cheng_color, label='Cheng et al. attack rate', linewidth=1)
# ax2.plot(covsyn_attack_rate_ticks, household_mean_attack_rate, 's-', 
#         color=attack_covsyn_color, label='CovSyn attack rate', linewidth=1)

# # Plot attack rate uncertainty
# ax2.errorbar(cheng_attack_rate_ticks, household_attack_rate, 
#             yerr=[household_attack_rate - household_attack_rate_lb, 
#                  household_attack_rate_upper_bound - household_attack_rate],
#             fmt='none', ecolor=attack_cheng_color, capsize=5, alpha=0.7, linewidth=1, markeredgewidth=1)

# ax2.errorbar(covsyn_attack_rate_ticks, household_mean_attack_rate, 
#             yerr=[household_mean_attack_rate - household_attack_lb, 
#                  household_attack_ub - household_mean_attack_rate],
#             fmt='none', ecolor=attack_covsyn_color, capsize=5, alpha=0.7, linewidth=1, markeredgewidth=1)
# # Set right y to red
# ax2.spines['right'].set_color(current_palette[2])
# ax2.tick_params(axis='y', colors=current_palette[2])
# ax2.yaxis.label.set_color(current_palette[2])

# # Set labels and ticks
# ax1.set_xlabel('Days from onset to first exposure')
# ax1.set_ylabel('Number of contacts')
# ax2.set_ylabel('Clinical attack rate (%)')
# ax1.set_xticks(x)
# ax1.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])



# # Set limits for Household layer
# # ax1.set_ylim(0.01, 1000)
# # ax2.set_ylim(0.01, 75)

# # Add grid for readability
# ax1.grid(axis='y', linestyle='--', alpha=0.3)

# # # Create legend
# # lines1, labels1 = ax1.get_legend_handles_labels()
# # lines2, labels2 = ax2.get_legend_handles_labels()
# # ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right', numpoints=1)

# # ax1.set_yscale('log')
# # ax2.set_yscale('log')
# # # Adjust layout
# # plt.tight_layout()

In [ ]:
# # Plot 1: Secondary infected cases vs close contacts comparison
# fig, ax1 = plt.subplots(figsize=(8,6))
# ax2 = ax1.twinx()

# # Define x positions
# x = np.arange(6)
# width = 0.15

# # Define colors (ColorBrewer palette)
# secondary_obs_color = "#b2df8a"    # Light green for observed secondary cases
# secondary_syn_color = "#33a02c"    # Green for synthetic secondary cases
# contact_obs_color = "#a6cee3"      # Light blue for observed contacts
# contact_syn_color = "#1f78b4"      # Blue for synthetic contacts

# # Define bar positions
# obs_secondary_x = x - 2*width
# syn_secondary_x = x - width
# obs_contact_x = x + width
# syn_contact_x = x + 2*width

# # Plot bars - Secondary infected cases (left y-axis)
# ax1.bar(obs_secondary_x, household_infected_contacts, width=width, 
#        color=secondary_obs_color, label='Observed secondary infection', alpha=0.9)
# ax1.bar(syn_secondary_x, household_mean_infection_case, width=width,
#        color=secondary_syn_color, label='CovSyn secondary infection', alpha=0.9)

# # Plot uncertainty for CovSyn secondary cases
# y_lower = household_mean_infection_case - household_infection_lb
# y_upper = household_infection_ub - household_mean_infection_case
# y_upper[y_upper < 0] = 0
# ax1.errorbar(syn_secondary_x, household_mean_infection_case, 
#             yerr=[y_lower, y_upper], fmt='.k', ecolor='black', 
#             capsize=1, linewidth=1, markeredgewidth=1, markersize=3)  # Smaller marker size

# # Plot bars - Close contacts (right y-axis)
# ax2.bar(obs_contact_x, household_contacts, width=width, 
#        color=contact_obs_color, label='Observed close contacts', alpha=0.9)
# ax2.bar(syn_contact_x, household_mean_contact_number, width=width,
#        color=contact_syn_color, label='CovSyn close contacts', alpha=0.9)

# # Plot uncertainty for CovSyn contacts
# y_lower = household_mean_contact_number - household_contact_lb
# y_upper = household_contact_ub - household_mean_contact_number
# ax2.errorbar(syn_contact_x, household_mean_contact_number, 
#             yerr=[y_lower, y_upper], fmt='.k', ecolor='black', 
#             capsize=1, linewidth=1, markeredgewidth=1, markersize=3)  # Smaller marker size

# # Style the axes
# ax1.set_xlabel('Days from onset to first exposure')
# ax1.set_ylabel('Household secondary infected cases')
# ax2.set_ylabel('Household close contacts')
# ax1.set_xticks(x)
# ax1.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])

# # Set colors for y-axes to match their data
# ax1.spines['left'].set_color(secondary_syn_color)
# ax1.tick_params(axis='y', colors=secondary_syn_color)
# ax1.yaxis.label.set_color(secondary_syn_color)

# ax2.spines['right'].set_color(contact_syn_color)
# ax2.tick_params(axis='y', colors=contact_syn_color)
# ax2.yaxis.label.set_color(contact_syn_color)

# # Set appropriate limits for household layer
# ax1.set_ylim(0, 7.5)  # For secondary cases
# ax2.set_ylim(0, 105)  # For contacts


# # Create combined legend
# lines1, labels1 = ax1.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', 
#           frameon=True)



In [ ]:
# # Plot 2: Attack rate comparison
# fig, ax = plt.subplots(figsize=(8, 6))

# # Define x positions
# x = np.arange(6)
# width = 0.25

# # Define colors
# obs_color = "#ff7f00"  # Light red for observed data
# syn_color = "#e41a1c"  # Red for synthetic data

# # Define positions
# obs_x = x - 0.15
# syn_x = x + 0.15

# # Plot attack rates with improved styling
# ax.plot(obs_x, household_attack_rate, 'o--', 
#        color=obs_color, label='Observed attack rate', linewidth=1.5, markersize=6)
# ax.plot(syn_x, household_mean_attack_rate, 's--', 
#        color=syn_color, label='CovSyn attack rate', linewidth=1.5, markersize=6)

# # Plot uncertainty with reduced capsize for cleaner appearance
# ax.errorbar(obs_x, household_attack_rate, 
#            yerr=[household_attack_rate - household_attack_rate_lb, 
#                 household_attack_rate_upper_bound - household_attack_rate],
#            fmt='none', ecolor=obs_color, capsize=5, alpha=0.8, linewidth=1)

# ax.errorbar(syn_x, household_mean_attack_rate, 
#            yerr=[household_mean_attack_rate - household_attack_lb, 
#                 household_attack_ub - household_mean_attack_rate],
#            fmt='none', ecolor=syn_color, capsize=5, alpha=0.8, linewidth=1)

# # Set labels and ticks
# ax.set_xlabel('Days from onset to first exposure')
# ax.set_ylabel('Household clinical attack rate (%)')
# ax.set_xticks(x)
# ax.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])

# # Set appropriate y-limit for household attack rate
# ax.set_ylim(-0.5, 100.5)

# # Add grid for better readability
# ax.grid(axis='y', linestyle='--', alpha=0.3)

# # Add legend
# ax.legend(loc='upper left', frameon=True, numpoints=1)


In [ ]:
# # Combined plot with contacts/infections and attack rate
# fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(12, 6))
# ax2 = ax1.twinx()

# # Define x positions
# x = np.arange(6)
# width = 0.15

# # Define colors (ColorBrewer palette)
# # Left plot colors
# secondary_obs_color = "#b2df8a"    # Light green for observed secondary cases
# secondary_syn_color = "#33a02c"    # Green for synthetic secondary cases
# contact_obs_color = "#a6cee3"      # Light blue for observed contacts
# contact_syn_color = "#1f78b4"      # Blue for synthetic contacts

# # Right plot colors
# obs_attack_color = "#ff7f00"       # Orange for observed attack rate
# syn_attack_color = "#e41a1c"       # Red for synthetic attack rate

# # Define bar positions
# obs_secondary_x = x - 2*width
# syn_secondary_x = x - width
# obs_contact_x = x + width
# syn_contact_x = x + 2*width

# # Left plot: Contacts & Secondary cases
# # Plot bars - Secondary infected cases (left y-axis)
# ax1.bar(obs_secondary_x, household_infected_contacts, width=width, 
#        color=secondary_obs_color, label='Observed secondary infection', alpha=0.9)
# ax1.bar(syn_secondary_x, household_mean_infection_case, width=width,
#        color=secondary_syn_color, label='CovSyn secondary infection', alpha=0.9)

# # Plot uncertainty for CovSyn secondary cases
# y_lower = household_mean_infection_case - household_infection_lb
# y_upper = household_infection_ub - household_mean_infection_case
# y_upper[y_upper < 0] = 0
# ax1.errorbar(syn_secondary_x, household_mean_infection_case,
#             yerr=[y_lower, y_upper], fmt='.k', ecolor='black',
#             capsize=1, linewidth=1, markeredgewidth=1, markersize=3)  # Smaller marker size

# # Plot bars - Close contacts (right y-axis)
# ax2.bar(obs_contact_x, household_contacts, width=width,
#        color=contact_obs_color, label='Observed close contacts', alpha=0.9)
# ax2.bar(syn_contact_x, household_mean_contact_number, width=width,
#        color=contact_syn_color, label='CovSyn close contacts', alpha=0.9)

# # Plot uncertainty for CovSyn contacts
# y_lower = household_mean_contact_number - household_contact_lb
# y_upper = household_contact_ub - household_mean_contact_number
# ax2.errorbar(syn_contact_x, household_mean_contact_number,
#             yerr=[y_lower, y_upper], fmt='.k', ecolor='black',
#             capsize=1, linewidth=1, markeredgewidth=1, markersize=3)  # Smaller marker size

# # Style the left plot axes
# ax1.set_xlabel('Days from onset to first exposure')
# ax1.set_ylabel('Household secondary infected cases')
# ax2.set_ylabel('Household close contacts')
# ax1.set_xticks(x)
# ax1.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])

# # Set colors for y-axes to match their data
# ax1.spines['left'].set_color(secondary_syn_color)
# ax1.tick_params(axis='y', colors=secondary_syn_color)
# ax1.yaxis.label.set_color(secondary_syn_color)

# ax2.spines['right'].set_color(contact_syn_color)
# ax2.tick_params(axis='y', colors=contact_syn_color)
# ax2.yaxis.label.set_color(contact_syn_color)

# # Set appropriate limits for household layer
# ax1.set_ylim(0, 7.5)  # For secondary cases
# ax2.set_ylim(0, 105)  # For contacts

# # Right plot: Attack rate
# # Define positions for attack rate plot
# obs_x = x - 0.15
# syn_x = x + 0.15

# # Plot attack rates with improved styling
# ax3.plot(obs_x, household_attack_rate, 'o--', 
#        color=obs_attack_color, label='Observed attack rate', linewidth=1.5, markersize=6)
# ax3.plot(syn_x, household_mean_attack_rate, 's--', 
#        color=syn_attack_color, label='CovSyn attack rate', linewidth=1.5, markersize=6)

# # Plot uncertainty with reduced capsize for cleaner appearance
# ax3.errorbar(obs_x, household_attack_rate, 
#            yerr=[household_attack_rate - household_attack_rate_lb, 
#                 household_attack_rate_upper_bound - household_attack_rate],
#            fmt='none', ecolor=obs_attack_color, capsize=5, alpha=0.8, linewidth=1)

# ax3.errorbar(syn_x, household_mean_attack_rate, 
#            yerr=[household_mean_attack_rate - household_attack_lb, 
#                 household_attack_ub - household_mean_attack_rate],
#            fmt='none', ecolor=syn_attack_color, capsize=5, alpha=0.8, linewidth=1)

# # Set labels and ticks for right plot
# ax3.set_xlabel('Days from onset to first exposure')
# ax3.set_ylabel('Household clinical attack rate (%)')
# ax3.set_xticks(x)
# ax3.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])

# # Set appropriate y-limit for household attack rate
# ax3.set_ylim(-0.5, 100.5)

# # Add grid for better readability
# ax1.grid(axis='y', linestyle='--', alpha=0.3)
# ax3.grid(axis='y', linestyle='--', alpha=0.3)

# # Create legends
# lines1, labels1 = ax1.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', frameon=True)

# ax3.legend(loc='upper left', frameon=True, numpoints=1)

# # Adjust layout
# plt.tight_layout()

In [ ]:
# Create a 3x2 grid for the complete figure comparing all layers
fig, axes = plt.subplots(3, 2, figsize=(12.5, 11))

# Define plotting function to reuse code across layers
def plot_layer_comparison(layer_name, contact_data, cheng_data, row_idx):
    # Unpack CovSyn data
    mean_contact, contact_lb, contact_ub, mean_infection, infection_lb, infection_ub, \
    mean_attack, attack_lb, attack_ub = contact_data
    
    # Unpack Cheng data
    contacts, infected_contacts, attack_rate, attack_lb_cheng, attack_ub_cheng = cheng_data
    
    # Get axes for this row
    ax1 = axes[row_idx, 0]
    ax2 = ax1.twinx()
    ax3 = axes[row_idx, 1]
    
    # Define x positions and widths
    x = np.arange(6)
    width = 0.15
    
    # Define colors based on layer
    secondary_obs_color = "#b2df8a"  # Light green for observed secondary
    secondary_syn_color = "#33a02c"  # Green for synthetic secondary
    contact_obs_color = "#a6cee3"    # Light blue for observed contacts
    contact_syn_color = "#1f78b4"    # Blue for synthetic contacts
    obs_attack_color = "#ff7f00"     # Orange for observed attack rate
    syn_attack_color = "#e41a1c"     # Red for synthetic attack rate
    
    # Define bar positions
    obs_secondary_x = x - 2*width
    syn_secondary_x = x - width
    obs_contact_x = x + width
    syn_contact_x = x + 2*width
    
    # Left plot: Contacts & Secondary cases
    # Plot bars - Secondary infected cases (left y-axis)
    ax1.bar(obs_secondary_x, infected_contacts, width=width, 
           color=secondary_obs_color, label='Observed secondary infection', alpha=0.9)
    ax1.bar(syn_secondary_x, mean_infection, width=width,
           color=secondary_syn_color, label='CovSyn secondary infection', alpha=0.9)
    
    # Plot uncertainty for CovSyn secondary cases
    y_lower = mean_infection - infection_lb
    y_upper = infection_ub - mean_infection
    y_upper[y_upper < 0] = 0
    ax1.errorbar(syn_secondary_x, mean_infection,
                yerr=[y_lower, y_upper], fmt='.k', ecolor='black',
                capsize=1, linewidth=1, markeredgewidth=1, markersize=3)
    
    # Plot bars - Close contacts (right y-axis)
    ax2.bar(obs_contact_x, contacts, width=width,
           color=contact_obs_color, label='Observed close contacts', alpha=0.9)
    ax2.bar(syn_contact_x, mean_contact, width=width,
           color=contact_syn_color, label='CovSyn close contacts', alpha=0.9)
    
    # Plot uncertainty for CovSyn contacts
    y_lower = mean_contact - contact_lb
    y_upper = contact_ub - mean_contact
    ax2.errorbar(syn_contact_x, mean_contact,
                yerr=[y_lower, y_upper], fmt='.k', ecolor='black',
                capsize=1, linewidth=1, markeredgewidth=1, markersize=3)
    
    # Style the left plot axes
    ax1.set_xlabel('Days from onset to first exposure')
    ax1.set_ylabel(f'{layer_name} secondary infected cases')
    ax2.set_ylabel(f'{layer_name} close contacts')
    ax1.set_xticks(x)
    ax1.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])
    
    # Set colors for y-axes to match their data
    ax1.spines['left'].set_color(secondary_syn_color)
    ax1.tick_params(axis='y', colors=secondary_syn_color)
    ax1.yaxis.label.set_color(secondary_syn_color)
    
    ax2.spines['right'].set_color(contact_syn_color)
    ax2.tick_params(axis='y', colors=contact_syn_color)
    ax2.yaxis.label.set_color(contact_syn_color)
    
    # Set appropriate limits based on layer
    if layer_name == 'Household':
        ax1.set_ylim(0, 8)    # For secondary cases
        ax2.set_ylim(0, 120)    # For contacts
    elif layer_name == 'Health care':
        ax1.set_ylim(0, 7)    # For secondary cases
        ax2.set_ylim(0, 350)    # For contacts
    elif layer_name == 'All':
        ax1.set_ylim(0, 30)    # For secondary cases
        ax2.set_ylim(0, 1400)   # For contacts
    
    # Add grid for better readability
    # ax1.grid(axis='y', linestyle='--', alpha=0.3)
    
    # Create legend for first row only to avoid repetition
    if row_idx == 0:
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', frameon=True)
    
    # Right plot: Attack rate
    # Define positions for attack rate plot
    obs_x = x - 0.15
    syn_x = x + 0.15
    
    # Plot attack rates with improved styling
    ax3.plot(obs_x, attack_rate, 'o--', 
           color=obs_attack_color, label='Observed data', linewidth=1.5, markersize=6)
    ax3.plot(syn_x, mean_attack, 's--', 
           color=syn_attack_color, label='CovSyn', linewidth=1.5, markersize=6)
    
    # Plot uncertainty with reduced capsize for cleaner appearance
    ax3.errorbar(obs_x, attack_rate, 
               yerr=[attack_rate - attack_lb_cheng, 
                    attack_ub_cheng - attack_rate],
               fmt='none', ecolor=obs_attack_color, capsize=1, alpha=0.8, linewidth=1)
    
    ax3.errorbar(syn_x, mean_attack, 
               yerr=[mean_attack - attack_lb, 
                    attack_ub - mean_attack],
               fmt='none', ecolor=syn_attack_color, capsize=1, alpha=0.8, linewidth=1)
    
    # Set labels and ticks for right plot
    ax3.set_xlabel('Days from onset to first exposure')
    ax3.set_ylabel(f'{layer_name} secondary attack rate (%)')
    ax3.set_xticks(x)
    ax3.set_xticklabels(['<0', '0-3', '4-5', '6-7', '8-9', '>9'])
    
    # Set appropriate y-limit based on layer
    if layer_name == 'Household':
        # ax3.set_ylim(0, 100)
        ax3.set_ylim(0, 25)
    elif layer_name == 'Health care':
        # ax3.set_ylim(0, 20)
        ax3.set_ylim(0, 5)
    elif layer_name == 'All':
        # ax3.set_ylim(0, 5)
        ax3.set_ylim(0, 2)
    
    # Add grid for right plot
    # ax3.grid(axis='y', linestyle='--', alpha=0.3)
    
    # Add legend for first row only
    if row_idx == 0:
        ax3.legend(loc='upper right', frameon=True, numpoints=1)

# Call the function for each layer
# Household layer (row 0)
plot_layer_comparison(
    'Household',
    (household_mean_contact_number, household_contact_lb, household_contact_ub,
     household_mean_infection_case, household_infection_lb, household_infection_ub,
     household_mean_attack_rate, household_attack_lb, household_attack_ub),
    (household_contacts, household_infected_contacts, household_attack_rate,
     household_attack_rate_lb, household_attack_rate_upper_bound),
    0
)

# Health care layer (row 1)
plot_layer_comparison(
    'Health care',
    (health_care_mean_contact_number, health_care_contact_lb, health_care_contact_ub,
     health_care_mean_infection_case, health_care_infection_lb, health_care_infection_ub,
     health_care_mean_attack_rate, health_care_attack_lb, health_care_attack_ub),
    (health_care_contacts, health_care_infected_contacts, health_care_attack_rate,
     health_care_attack_rate_lb, health_care_attack_rate_upper_bound),
    1
)

# All layers (row 2)
plot_layer_comparison(
    'All',
    (all_mean_contact_number, all_contact_lb, all_contact_ub,
     all_mean_infection_case, all_infection_lb, all_infection_ub,
     all_mean_attack_rate, all_attack_lb, all_attack_ub),
    (all_contacts, all_infected_contacts, all_attack_rate,
     all_attack_rate_lb, all_attack_rate_upper_bound),
    2
)

# Add A, B, C, D, E, F subplot labels
# subplot_labels = ['A', 'B', 'C', 'D', 'E', 'F']
# for i, ax in enumerate(axes.flat):
#     ax.text(-0.15, 1.05, subplot_labels[i], transform=ax.transAxes, 
#             fontsize=16, fontweight='bold', va='top')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(hspace=0.3)  # Increase spacing between rows

# Save figure
# plt.savefig('RW2025_covsyn_contact_bar_and_attack_rate.pdf', bbox_inches='tight')

## 1.3 Plot the detail contact date and infection data bar char

In [ ]:
contact_day_shift, exposure_day_array, daily_contact_array, infection_day_shift, daily_infection_array, daily_secondary_attack_rate = \
    generate_contact_day_vs_infection_day_array(course_of_disease_data_all[0:1], contact_data_all[0:1])
print(f'contact_day_shift: {contact_day_shift}')
print(f'exposure_day_array: {exposure_day_array}')
print(f'daily_contact_array: {daily_contact_array}')
print(f'infection_day_shift: {infection_day_shift}')
print(f'daily_infection_array: {daily_infection_array}')
print(f'daily_secondary_attack_rate: {daily_secondary_attack_rate}')

In [ ]:
save_fig = False
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=10, monte_carlo_number=100, num_source_cases=100, layer='Household', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=5, monte_carlo_number=100, num_source_cases=100, layer='School', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=5, monte_carlo_number=100, num_source_cases=100, layer='Workplace', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=5, monte_carlo_number=100, num_source_cases=100, layer='Health care', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=2, monte_carlo_number=100, num_source_cases=100, layer='Municipality', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=2, monte_carlo_number=100,num_source_cases=100, layer='All', save_fig=save_fig)

## 1.4 Plot logistic contact p example

In [ ]:
def generate_contact_p(t, ps, ph, s, ts, tr):
    y = ph + (ph-ps) - (ph-ps)/(1+np.exp(-s*(t-ts))) - (ph-ps)/(1+np.exp(s*(t-tr)))

    return(y)

ps, ph, s, ts, tr = 0.4, 0.8, 2, 5, 15
t = np.arange(-10, 50, 0.1)
y = generate_contact_p(t, ps, ph, s, ts, tr)
plt.figure(figsize=(8, 2))
plt.plot(t, y)
plt.xlim([0, 20])
plt.ylim([0.3, 0.9])
plt.xticks(np.array([0, ts, tr]), ['$\mathrm{t_{I^S}}$', '$t_{PS}$', '$t_{PN}$'])
plt.yticks(np.array([ps, ph]), ['$p_S$', '$p_H$'])
plt.grid()
plt.savefig('RW2022_contact_p.pdf')

In [ ]:
def generate_logistic_contact_p(t, healthy_p, symptom_p, steepness, symptom_phase, width):
        # generate_logistic_contact_p: generate contact probability on day t
        #     healthy_p: Contact probability when healthy
        #     symptom_p: Contact probability when symptomatic
        #     steepness: Steepness of the logistic function
        #     symptom_phase: Phase relative to symptom-onset for symptomatic (days)
        #     width: Days different between symptom phase and normal phase
        normal_phase = symptom_phase + width
        logistic_p = healthy_p + (healthy_p-symptom_p) - (healthy_p-symptom_p)/(1+np.exp(-steepness*(
            t-symptom_phase))) - (healthy_p-symptom_p)/(1+np.exp(steepness*(t-normal_phase)))

        return (logistic_p)


contact_p, contact_previous_day_p, healthy_p, symptom_p = 0.1, 0.5, 0.1, 0.5
p = [contact_p, contact_previous_day_p, healthy_p, symptom_p]
steepness, symptom_phase, recover_phase = 10, 5, 2

t = np.arange(0, 50, 0.1)
logistic_p = generate_logistic_contact_p(t, healthy_p, symptom_p, steepness, symptom_phase, recover_phase)

plt.plot(t, logistic_p)
plt.grid()
plt.xlim([0, 20])

## 1.5 Plot contact number histogram

In [ ]:
contact_date_stack_matrix, index = generate_stack_contact_matrix(contact_data_all[0:100], layer='Household')
plt.hist(np.diff(index, prepend=0), bins=100)


In [ ]:
household_effective_contact_array = np.array([])
school_effective_contact_array = np.array([])
workplace_effective_contact_array = np.array([])
health_care_effective_contact_array = np.array([])
municipality_effective_contact_array = np.array([])
for i in range(100):
    if np.nansum(contact_data_all[i]['household_effective_contacts_infection_time']) > 0:
        household_effective_contact_array = np.append(household_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['household_effective_contacts_infection_time']))))
    else:
        household_effective_contact_array = np.append(
            household_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['school_effective_contacts_infection_time']) > 0:
        school_effective_contact_array = np.append(school_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['school_effective_contacts_infection_time']))))
    else:
        school_effective_contact_array = np.append(
            school_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['workplace_effective_contacts_infection_time']) > 0:
        workplace_effective_contact_array = np.append(workplace_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['workplace_effective_contacts_infection_time']))))
    else:
        workplace_effective_contact_array = np.append(
            workplace_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['health_care_effective_contacts_infection_time']) > 0:
        health_care_effective_contact_array = np.append(health_care_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['health_care_effective_contacts_infection_time']))))
    else:
        health_care_effective_contact_array = np.append(
            health_care_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['municipality_effective_contacts_infection_time']) > 0:
        municipality_effective_contact_array = np.append(municipality_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['municipality_effective_contacts_infection_time']))))
    else:
        municipality_effective_contact_array = np.append(
            municipality_effective_contact_array, 0)

all_effective_contact_array = household_effective_contact_array + school_effective_contact_array + \
    workplace_effective_contact_array + health_care_effective_contact_array + municipality_effective_contact_array


In [ ]:
plt.hist(all_effective_contact_array)